<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

In [2]:
pip install keras==2.3.1

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')
trn=trn.drop(['isFraud'],1)

In [6]:
ls=list(trn.filter(regex='V'))
trn=trn.drop(ls,1)
tst=tst.drop(ls,1)
trn.shape

(590540, 54)

In [7]:
import gc
fnl=pd.concat([trn,tst],0)
fnl=fnl.reset_index(drop=True)
del([trn,tst])
gc.collect()

11

In [8]:
fnl['nan_count']=fnl.isna().sum(1)
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
for col in tqdm(list(fnl.select_dtypes(include=object))):
    fnl[col]=fnl[col].fillna('Other')
    fnl[col]=LabelEncoder().fit_transform(fnl[col].astype(str))

100%|██████████| 14/14 [00:05<00:00,  2.56it/s]


In [0]:
fnl=fnl.drop(['TransactionDT','TransactionID'],1)
def mape(data,ls):
    return [data if data in ls else 'Other']

def handle_cont(df,cols):
  for col in cols:
    if df[col].skew()>1:
        df[col]=np.log10(df[col]+1-min(0,min(df[col].fillna(0))))
    mn=df[col].mean()
    std=df[col].std()
    df[col]=(df[col]-mn)/(std+1e-6)
    df[col]=df[col].fillna(0)
  return df[cols]
disc=['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']
cont=[i for i in fnl.columns if i not in disc]
fnl_cont=handle_cont(fnl,cont)

In [10]:
def handle_disc(df,cols,num):
  df=df[cols]
  for col in cols:
    df[col]=df[col].astype(str)
    par=min(df[col].nunique(),num)
    vc=list(df[col].value_counts().iloc[:par-1].index)+['Other']
    print(df[col].nunique())
    df[col]=df[col].apply(lambda x:x if x in vc else 'Other')
    print(df[col].nunique())
    print()
    dum=pd.get_dummies(df[col])
    ls=[]
    for i in dum.columns:
      ls.append(str(col)+str(i)+'_dum')
    dum.columns=ls
    df=pd.concat([df,dum],1)
    del([dum])
    gc.collect()
  df=df.drop(cols,1)
  sp=df.shape[1]
  return df
fnl_disc=handle_disc(fnl,disc,140)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


5

17091
140

502
140

134
134

5
5

139
139

5
5

442
140

94
94

61
61

61
61

3
3

3
3

3
3

4
4

3
3

3
3

3
3

3
3

3
3



In [11]:
z=fnl_cont.isna().sum()
z[z>0]


Series([], dtype: int64)

In [12]:
z=fnl_disc.isna().sum()
z[z>0]

Series([], dtype: int64)

In [13]:
fnl=pd.concat([fnl_disc,fnl_cont],1)
import gc
fnl=fnl.fillna(0)
del([fnl_disc,fnl_cont])
gc.collect()

0

In [0]:
cat=list(fnl.filter(regex='dum'))
cat_shape=len(list(fnl.filter(regex='dum')))
num_shape=len([i for i in fnl.columns if i not in cat])

In [15]:
import keras
import random
import tensorflow as tf
import keras.backend as K

from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.utils.generic_utils import get_custom_objects
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

np.random.seed(42) # NumPy
random.seed(42) # Python
# Compatible with tensorflow backend

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})



from keras.utils import Sequence
class DAESequence(Sequence):
    def __init__(self,df,no_dum,frac=0.15,dumm=range(911),batch_size=2048):
        self.batch_size=batch_size
        self.frac=0.15
        self.dumm=dumm
        self.df=df
        self.cat_data=df[dumm].values
        self.num_data=df[no_dum].values
        self.no_dumm=no_dum
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df
        self.idx=[]
        
        
        
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    
    
    
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        
        
        inps=[]
        outs=[]
        output_x=self.data.iloc[idx:last]
        
        data=output_x[self.no_dumm].values
        noise_x=data.copy()
        for i in range(len(self.no_dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=self.num_data[frm,i]
            
        inps.append(noise_x)
        outs.append(data)
        
        data=output_x[self.dumm].values
        noise_x=data.copy()
        for i in range(len(self.dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=self.cat_data[frm,i]
        
        
        
        inps.append(noise_x)
        outs.append(data)
        
        
        
        
        
        return inps,outs





batch_size=2048
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
auto_ckpt = ModelCheckpoint("ae.model", monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min', period=1)
from keras import backend as K


class WarmUpLearningRateScheduler(keras.callbacks.Callback):
    """Warmup learning rate scheduler
    """

    def __init__(self, warmup_batches, init_lr, verbose=0):
        """Constructor for warmup learning rate scheduler

        Arguments:
            warmup_batches {int} -- Number of batch for warmup.
            init_lr {float} -- Learning rate after warmup.

        Keyword Arguments:
            verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpLearningRateScheduler, self).__init__()
        self.warmup_batches = warmup_batches
        self.init_lr = init_lr
        self.verbose = verbose
        self.batch_count = 0
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.batch_count = self.batch_count + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        if self.batch_count <= self.warmup_batches:
            lr = self.batch_count*self.init_lr/self.warmup_batches
            K.set_value(self.model.optimizer.lr, lr)
            if self.verbose > 0:
                print('\nBatch %05d: WarmUpLearningRateScheduler setting learning '
                      'rate to %s.' % (self.batch_count + 1, lr))
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.001)
import gc
gc.collect()




Using TensorFlow backend.


59

In [0]:
from keras.layers import concatenate
K.clear_session()
from keras.optimizers import Adam
from keras import regularizers
from keras.regularizers import l2 
def create_model():
    num_inp = Input(shape=(num_shape,))
    cat_inp = Input(shape=(cat_shape,))
    inps = concatenate([num_inp, cat_inp])
    x = Dense(512, activation=custom_gelu)(inps)
    x = Dense(256, activation=custom_gelu)(x)
    x = Dense(512, activation = custom_gelu)(x)
    x = Dropout(.2)(x)
    cat_out = Dense(cat_shape, activation = "linear")(x)
    num_out = Dense(num_shape, activation = "linear")(x)
    model = Model(inputs=[num_inp, cat_inp], outputs=[num_out, cat_out])
    model.compile(
        optimizer=Adam(.05, clipnorm = 1, clipvalue = 1),
        loss=["mse", "mse"]
    )
    return model









In [17]:

import gc

from math import *
gc.collect()
epochs = 10
num=[i for i in fnl.columns if i not in cat]
train_gen=DAESequence(fnl,num,dumm=cat,batch_size=batch_size)
model_mse = create_model()
hist = model_mse.fit_generator(train_gen, steps_per_epoch=len(fnl)//batch_size, epochs=epochs,
                           verbose=1, workers=-1, 
                           use_multiprocessing=True,
                              callbacks=[auto_ckpt, warm_up_lr])

Epoch 1/10
535/535 [==============================] - 101s 188ms/step - loss: 0.2098 - dense_5_loss: 0.1988 - dense_4_loss: 0.0111

Epoch 00001: loss improved from inf to 0.20982, saving model to ae.model
Epoch 2/10
535/535 [==============================] - 100s 186ms/step - loss: 0.0807 - dense_5_loss: 0.0751 - dense_4_loss: 0.0055

Epoch 00002: loss improved from 0.20982 to 0.08066, saving model to ae.model
Epoch 3/10
535/535 [==============================] - 101s 188ms/step - loss: 0.0720 - dense_5_loss: 0.0671 - dense_4_loss: 0.0050

Epoch 00003: loss improved from 0.08066 to 0.07204, saving model to ae.model
Epoch 4/10
535/535 [==============================] - 100s 188ms/step - loss: 0.0685 - dense_5_loss: 0.0637 - dense_4_loss: 0.0048

Epoch 00004: loss improved from 0.07204 to 0.06851, saving model to ae.model
Epoch 5/10
535/535 [==============================] - 106s 198ms/step - loss: 0.0650 - dense_5_loss: 0.0604 - dense_4_loss: 0.0046

Epoch 00005: loss improved from 0.06

In [18]:
model_mse.layers

In [0]:

path = F"/content/gdrive/My Drive/autoenc_final.hdf5" 
model_mse.save_weights(path)

In [20]:
from tqdm import tqdm
your_new_df=pd.DataFrame()
for i in tqdm([4,5,6]):
    intermediate_layer_model = Model(inputs=model_mse.input, outputs=model_mse.layers[i].output)
    z = pd.DataFrame(intermediate_layer_model.predict([fnl[num],fnl[cat]]))
    columns_names = ['Hidden_'+str(i)+'_'+str(l) for l in range(0, z.shape[1])]
    z.columns=columns_names
    your_new_df=pd.concat([your_new_df,z],1)

100%|██████████| 3/3 [04:10<00:00, 83.38s/it]


In [21]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
your_new_df=reduce_mem_usage(your_new_df)


Memory usage of dataframe is 5357.57 MB
Memory usage after optimization is: 2678.79 MB
Decreased by 50.0%


In [0]:

trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
your_new_df=your_new_df.loc[:trn.shape[0]]
your_new_df['isFraud']=trn['isFraud']
fnl=pd.DataFrame()
cols=[]
val=[]

In [0]:

your_new_df=your_new_df.dropna()
your_new_df=your_new_df.replace([np.inf,-np.inf],0)

In [0]:
ls=list(your_new_df)
ls.remove('isFraud')

In [0]:
your_new_df=your_new_df.drop(['isFraud'],1)
path = F"/content/gdrive/My Drive/extra__1.pkl" 
your_new_df.to_pickle(path)